In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv
/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip
/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip


In [2]:
# Sentiment Analysis on Movie Reviews 대회
train = pd.read_table("/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip")
test = pd.read_table("/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip")


In [3]:
display(train, test)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
...,...,...,...
66287,222348,11855,"A long-winded , predictable scenario ."
66288,222349,11855,"A long-winded , predictable scenario"
66289,222350,11855,"A long-winded ,"
66290,222351,11855,A long-winded


In [4]:
all_data = pd.concat([train, test])
all_data["Phrase"].iloc[0]
#머신러닝으로는 단어단위로만 의미파악. 딥러닝은 문맥,맥락적 의미 파악이 가능
# 딥러닝 돌리기 전에 먼저 텍스트 전처리

'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'

In [5]:
# 토크나이저 선언 (케라스의 전처리 도구)
from keras.preprocessing.text import Tokenizer
tk = Tokenizer()
# 정수인코딩

In [6]:
# 코퍼스의 모든 단어를 숫자로 매핑
tk.fit_on_texts(all_data["Phrase"])
tk.word_index # 빈도순
all_text = tk.texts_to_sequences(all_data["Phrase"]) # 각 텍스트를 숫자로
all_data["Phrase"].iloc[0]
all_text[0]

[2,
 315,
 3,
 16573,
 7660,
 1,
 8313,
 9,
 53,
 8,
 47,
 13,
 1,
 3940,
 8,
 187,
 47,
 13,
 1,
 13024,
 61,
 3,
 89,
 592,
 12156,
 19,
 617,
 3,
 89,
 2810,
 5,
 52,
 3,
 2,
 42]

In [7]:
# 패딩 (딥러닝 돌리기 전에 데이터의 차원(단어의 개수)이 모두 동일하게 맞춰줘야함)
from keras.preprocessing.sequence import pad_sequences
all_pad = pad_sequences(all_text) #옵션 조정해서 패딩 종류 바꿀 수 있음 (디폴트는 앞쪽에 채워줌)
all_pad

array([[   0,    0,    0, ...,    3,    2,   42],
       [   0,    0,    0, ...,   13,    1, 3940],
       [   0,    0,    0, ...,    0,    2,  315],
       ...,
       [   0,    0,    0, ...,    2,  118, 4456],
       [   0,    0,    0, ...,    2,  118, 4456],
       [   0,    0,    0, ...,    0,  343, 1623]], dtype=int32)

In [8]:
all_pad.shape #가장 길었던 애가 단어 52개

(222352, 52)

In [9]:
all_pad[0] # 패딩이 0으로 들어간 이유는?

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     2,
         315,     3, 16573,  7660,     1,  8313,     9,    53,     8,
          47,    13,     1,  3940,     8,   187,    47,    13,     1,
       13024,    61,     3,    89,   592, 12156,    19,   617,     3,
          89,  2810,     5,    52,     3,     2,    42], dtype=int32)

In [10]:
train_pad = all_pad[:len(train)]
test_pad = all_pad[len(train):]

In [11]:
len(tk.word_index) # 단어의 총 개수

17780

In [12]:
all_data["Sentiment"].value_counts() # 정답값 클래스 5개

2.0    79582
3.0    32927
1.0    27273
4.0     9206
0.0     7072
Name: Sentiment, dtype: int64

In [13]:
from keras import Sequential # sequential, functional api 이렇게 두가지 있음
from keras.layers import *    #layer 생성 함수 모두 가져오기

#이제 각각의 layer를 설정해 쌓아준다.
model = Sequential()
model.add(Embedding(len(tk.word_index)+1,10,input_length=all_pad.shape[1]))
# len()+1 해주는데 +1은 padding 개수 추가해준것. 그냥 len()결과를 padding개수 포함안되어 있어서
# 10보다 이상으로 설정하면 학습이 느려질수 있다.

model.add(Flatten()) 
#flatten() 통해 embedding결과 dim이 (none 제외) 2차원으로 생성된 벡터를 1차원의 벡터로 줄여준다.

model.add(Dense(128, activation="relu"))
model.add(Dense(5,activation="softmax")) # 출력층 : 정답클래스개수, 활성함수(softmax : 0~1사이값)
model.compile(metrics=["acc"], optimizer="adam",loss="sparse_categorical_crossentropy")
# compile 선언하여 모델 완성시켜줌. 필요한 모듈들 선언해줌
# regression문제면, loss="categorical_crossentropy"로 선언해주면됨
# classifier문제는, loss="sparse_categorical_crossentropy" 로 선언

In [14]:
# 자연어 처리를 시작할 때 Embedding 해주고 시작해야한다.
# Embedding : 단어들의 의미를 파악하게 해주는 층 (밀집벡터에 임베딩)
# 옵션 3개 : 전체 단어 종류의 개수, 의미의 차원(자유롭게 조절), 데이터의 차원(길이)
# 단어들의 의미는 이제 파악을 했음 이제 학습할거니까 dense레이어 같은거 들어가야되는데..
# 임베딩층이 2차원인데 dense레이어는 1차원이라 1차원으로 만들어줘야함. 그래서 flatten 넣어줌
# sparse적는 이유? 회귀문제가 아니고 분류문제이기 때문

# Embedding층 더 자세히 알고 싶으면
# https://wikidocs.net/32105
# https://wikidocs.net/64779 파이토치
# https://wikidocs.net/22644

In [15]:
model.summary()
# 모델 시각화

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 52, 10)            177810    
_________________________________________________________________
flatten (Flatten)            (None, 520)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               66688     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
Total params: 245,143
Trainable params: 245,143
Non-trainable params: 0
_________________________________________________________________


In [16]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))
model.fit(train_pad,train["Sentiment"],batch_size=32,epochs=1)

4877/4877 [==============================] - 16s 3ms/step - loss: 1.0868 - acc: 0.5646


In [17]:
from keras import Sequential
from keras.layers import *
model = Sequential()
model.add(Embedding(len(tk.word_index)+1,10,input_length=all_pad.shape[1]))
model.add(LSTM(32)) #특징을 32개추출 
# 문맥적 정보를 파악하는 rnn 계열 : lstm, gru (차원축소까지 해줌 그래서 flatten안해줘도됨)

model.add(Dense(5,activation="softmax"))
model.compile(metrics=["acc"], optimizer="adam",loss="sparse_categorical_crossentropy")
model.fit(train_pad,train["Sentiment"],batch_size=256,epochs=10)

Epoch 1/10
610/610 [==============================] - 35s 54ms/step - loss: 1.2657 - acc: 0.5103
Epoch 2/10
610/610 [==============================] - 33s 55ms/step - loss: 0.9054 - acc: 0.6364
Epoch 3/10
610/610 [==============================] - 32s 53ms/step - loss: 0.7742 - acc: 0.6832
Epoch 4/10
610/610 [==============================] - 33s 55ms/step - loss: 0.7346 - acc: 0.6988
Epoch 5/10
610/610 [==============================] - 33s 55ms/step - loss: 0.7125 - acc: 0.7068
Epoch 6/10
610/610 [==============================] - 33s 55ms/step - loss: 0.6890 - acc: 0.7150
Epoch 7/10
610/610 [==============================] - 33s 54ms/step - loss: 0.6720 - acc: 0.7227
Epoch 8/10
610/610 [==============================] - 34s 56ms/step - loss: 0.6558 - acc: 0.7264
Epoch 9/10
610/610 [==============================] - 33s 55ms/step - loss: 0.6392 - acc: 0.7333
Epoch 10/10
610/610 [==============================] - 45s 73ms/step - loss: 0.6244 - acc: 0.7383


In [18]:
# 예측
result = model.predict(test_pad)
sub = pd.read_csv("/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv")
sub["Sentiment"] = result.argmax(1) # 가장 높은 확률값 갖는 걸 가져오기
sub.to_csv("subsub.csv", index=0)